In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.dates import AutoDateLocator
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score
from matplotlib.ticker import MultipleLocator
from matplotlib.cm import tab10
from itertools import combinations
from collections import defaultdict

In [ ]:
os.makedirs("new_cluster_results/daily", exist_ok=True)
os.makedirs("new_cluster_results/monthly", exist_ok=True)

In [ ]:
name_map = pd.read_csv("sewershed_names.csv")
name_map_dict = {k:v for k,v in zip(name_map.sewershed_result_name, name_map.cdphe_flow_name)}
name_map_dict_r = {v:k for k,v in name_map_dict.items()}

In [ ]:
# Read the filter file
filter_sewersheds = pd.read_csv("flow_mobile_corr_daily.csv", index_col=0)
# Modify the names to match the ones in the data files
filter_sewersheds["sewershed_result_name"] = filter_sewersheds["sewershed_result_name"].replace(name_map_dict)
# Make a set of the sewersheds for inclusion
use_sewersheds = set(filter_sewersheds.sewershed_result_name)
# Print size
print(f"Inclusion set has {len(use_sewersheds)} sewersheds.")

In [ ]:
# Path to monthly dataset
MONTHLY_VISITATION_JOINED = "/biostats_share/hillandr/data/WW_Mobility_2025_04_04/processed/2025_04_04_monthly_devices.csv"

In [ ]:
# Read monthly data
monthly_visitation = pd.read_csv(MONTHLY_VISITATION_JOINED)
# Create a summed device count
monthly_visitation["DEVICE_SUM"] = monthly_visitation["SOURCE_AREA_DEVICE_COUNT"] + monthly_visitation["NUMBER_DEVICES_RESIDING"]

In [ ]:
# Pivot the monthly data
monthly_data_training_format = monthly_visitation.pivot(index="AREA_SEWERSHED", columns=["DATE_RANGE_START"], values=["DEVICE_SUM"])
# Drop the unnecessary level
monthly_data_training_format.columns = monthly_data_training_format.columns.droplevel(level=0)
# Log transform
monthly_data_training_format_log = np.log(monthly_data_training_format)
# Z-Score normalize (with respect to the temporal axis for each series)
monthly_data_training_format_log_norm = monthly_data_training_format_log.sub(monthly_data_training_format_log.mean(axis=1), axis=0).div(monthly_data_training_format_log.std(axis=1), axis=0)
# Subset to just the 
monthly_data_training_format_log_norm = monthly_data_training_format_log_norm.loc[monthly_data_training_format_log_norm.index.isin(use_sewersheds)]
# Print size
print(f"Monthly dataset has {monthly_data_training_format_log_norm.shape[0]} sewersheds.")

In [ ]:
def get_silhouette(i: int, df: pd.DataFrame, n_reps:int = 50):
    rng = np.random.default_rng(42)
    km_list = []
    for seed in rng.integers(low=0, high=np.iinfo("int32").max, size=n_reps):
        km_obj = KMeans(i, random_state=seed)
        km_labels = km_obj.fit_predict(df)
        km_list.append(silhouette_score(df, km_labels))
    return pd.DataFrame({"Silhouette Score": km_list}, index=pd.RangeIndex(n_reps, name="rep_id"))

In [ ]:
monthly_k_to_ss = pd.concat({i:get_silhouette(i, monthly_data_training_format_log_norm) for i in range(2, 15)}, names="k").reset_index()
monthly_k_to_ss["flag"] = 1

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(data=monthly_k_to_ss, x="k", y="Silhouette Score", errorbar=("pi", 50), style="flag", markers=True, legend=False, ax=ax)
# ax.plot(monthly_k_to_ss, "o-")
ax.set_xlabel("K Clusters")
ax.set_ylabel("Silhouette Score")
ax.set_title("Monthly Silhouette Score for K=[2,14]")
ax.xaxis.set_minor_locator(MultipleLocator())
fig.savefig("new_cluster_results/monthly/monthly_silhouette_score.png")

In [ ]:
def plot_k(k: int, df: pd.DataFrame, filename: str = None):
    km_obj = KMeans(k, random_state=42)
    km_labels = km_obj.fit_predict(df)
    fig, ax = plt.subplots(nrows=k, figsize=(12,k*3))
    for i,unq_lbl in enumerate(np.unique(km_labels)):
        #print(f"Cluster {unq_lbl}:")
        #print(', '.join(data_training_format_log_norm.index[km_labels == i]))
        ax[i].plot(df.loc[km_labels == i].transpose(), alpha=0.3, color=tab10(i))
        ax[i].xaxis.set_major_locator(AutoDateLocator())
    if filename is not None:
        fig.savefig(filename)
    plt.close(fig)
    return km_labels

In [ ]:
monthly_k_2_labels = plot_k(2, monthly_data_training_format_log_norm, filename="new_cluster_results/monthly/monthly_kmeans_k2.png")
monthly_k_3_labels = plot_k(3, monthly_data_training_format_log_norm, filename="new_cluster_results/monthly/monthly_kmeans_k3.png")
monthly_k_9_labels = plot_k(9, monthly_data_training_format_log_norm, filename="new_cluster_results/monthly/monthly_kmeans_k9.png")

In [ ]:
monthly_clust_labels_df = pd.DataFrame({"kmeans_k2": monthly_k_2_labels, "kmeans_k3": monthly_k_3_labels, "kmeans_k9": monthly_k_9_labels}, index=monthly_data_training_format_log_norm.index)
monthly_clust_labels_df.to_csv("new_cluster_results/monthly/2025_06_25_monthly_kmeans_cluster_labels.csv")

# Daily Data

In [ ]:
# Path to daily dataset
DAILY_DATA = "/biostats_share/hillandr/data/WW_Mobility_2025_04_04/processed/2025_04_04_daily_visits_sum.csv"

In [ ]:
# Read daily visitation
daily_visitation = pd.read_csv(DAILY_DATA)

In [ ]:
# Pivot the daily data
daily_data_training_format = daily_visitation.pivot(index="AREA_SEWERSHED", columns=["DAY"], values=["STOPS_BY_DAY_L"])
# Drop the unnecessary level
daily_data_training_format.columns = daily_data_training_format.columns.droplevel(level=0)
# Log transform
daily_data_training_format_log = np.log(1+daily_data_training_format)
# Z-Score normalize (with respect to the temporal axis for each series)
daily_data_training_format_log_norm = daily_data_training_format_log.sub(daily_data_training_format_log.mean(axis=1), axis=0).div(daily_data_training_format_log.std(axis=1), axis=0)
# Subset to just the 
daily_data_training_format_log_norm = daily_data_training_format_log_norm.loc[daily_data_training_format_log_norm.index.isin(use_sewersheds)]
# Print size
print(f"Daily dataset has {daily_data_training_format_log_norm.shape[0]} sewersheds.")

In [ ]:
daily_k_to_ss = pd.concat({i:get_silhouette(i, daily_data_training_format_log_norm) for i in range(2, 15)}, names="k").reset_index()
daily_k_to_ss["flag"] = 1

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(data=daily_k_to_ss, x="k", y="Silhouette Score", errorbar=("pi", 50), style="flag", markers=True, legend=False, ax=ax)
#ax.plot(daily_k_to_ss, "o-")
ax.set_xlabel("K Clusters")
ax.set_ylabel("Silhouette Score")
ax.set_title("Daily Silhouette Score for K=[2,14]")
ax.xaxis.set_minor_locator(MultipleLocator())
fig.savefig("new_cluster_results/daily/daily_silhouette_score.png")

In [ ]:
daily_k_2_labels = plot_k(2, daily_data_training_format_log_norm, filename="new_cluster_results/daily/daily_kmeans_k2.png")
daily_k_3_labels = plot_k(3, daily_data_training_format_log_norm, filename="new_cluster_results/daily/daily_kmeans_k3.png")
daily_k_6_labels = plot_k(6, daily_data_training_format_log_norm, filename="new_cluster_results/daily/daily_kmeans_k6.png")

In [ ]:
daily_clust_labels_df = pd.DataFrame({"kmeans_k2": daily_k_2_labels, "kmeans_k3": daily_k_3_labels, "kmeans_k6": daily_k_6_labels}, index=monthly_data_training_format_log_norm.index)
daily_clust_labels_df.to_csv("new_cluster_results/daily/2025_06_25_daily_kmeans_cluster_labels.csv")

## Comparison to old results

In [ ]:
old_monthly_labels = pd.read_csv("../WW_mobility/kmeans_cluster_labels.csv")

In [ ]:
old_monthly_labels

In [ ]:
merged_labels = pd.merge(old_monthly_labels, monthly_clust_labels_df, on="AREA_SEWERSHED", how="inner", suffixes=["_old", "_new"], validate="1:1")

In [ ]:
adjusted_rand_score(merged_labels["kmeans_k2_old"], merged_labels["kmeans_k2_new"])

In [ ]:
#merged_labels.loc[merged_labels["kmeans_k2_old"] != merged_labels["kmeans_k2_new"]]

In [ ]:
!tar --exclude '*/.ipynb_checkpoints' -czvf 2025_06_30_new_cluster_results.tar.gz new_cluster_results/

In [ ]:
!rm -rf new_cluster_results/

## Monthly Stability at K=2

In [ ]:
rng = np.random.default_rng(42)
results_list = []
for seed in rng.integers(low=0, high=np.iinfo("int32").max, size=50):
    km_obj = KMeans(2,random_state=seed)
    km_labels = km_obj.fit_predict(monthly_data_training_format_log_norm)
    results_list.append(km_labels)

In [ ]:
fig, ax = plt.subplots()
ax.hist([adjusted_rand_score(x, y) for x,y in combinations(results_list, 2)]);